# Counting Methods

In [1]:
import numpy as np
import matplotlib.pyplot as plt

class Method():
    '''
    Base method class for solving the equations or counting errors
    '''
    
    def __init__(self, equation, x0, y0, x1, steps):
        self.equation = equation
        self.grid = self._init_grid(x0, y0, x1, steps)
        self._solve()
        
    def _init_grid(self, x0, y0, x1, steps):
        self.delta = (x1-x0)/steps
        x_grid = np.arange(x0, x1, self.delta)
        y_grid = np.array([y0])
        return [x_grid, y_grid]
        
    def _solve(self):
        raise NotImplementedError()
        
    def show(self, line='r-', linewidth=2):
        '''
        Displaying the method grid basssed on line parameters
        '''
        plt.plot(self.grid[0], self.grid[1], line, label=self.name, linewidth=linewidth)
        plt.xlabel('x')
        plt.ylabel('y(x)')
        plt.legend(loc='upper left')

In [2]:
class Euler(Method):
    '''
    Euler's method for solving initial value problem
    '''
    
    def __init__(self, equation, x0, y0, x1, steps):
        self.name = 'Euler'
        super().__init__(equation, x0, y0, x1, steps)
    
    def _solve(self):
        delta = self.delta
        equation = self.equation
        for x in self.grid[0][:-1]:
            y = self.grid[1][-1]
            
            self.grid[1] = np.append(self.grid[1], [y+delta*equation(x,y)])

In [3]:
class ImprovedEuler(Method):
    '''
    Improved euler's method for solving initial value problem
    '''
    
    def __init__(self, equation, x0, y0, x1, steps):
        self.name = 'Improved Euler'
        super().__init__(equation, x0, y0, x1, steps)
    
    def _solve(self):
        delta = self.delta
        equation = self.equation
        for x in self.grid[0][:-1]:
            y = self.grid[1][-1]
            
            m1 = equation(x, y)
            m2 = equation(x+delta, y+delta*m1)
            
            self.grid[1] = np.append(self.grid[1], [y+delta*(m1+m2)/2])

In [4]:
class RungeKutta(Method):
    '''
    Runge Kutta method for solving initial value problem
    '''
    
    def __init__(self, equation, x0, y0, x1, steps):
        self.name = 'Runge Kunkka'
        super().__init__(equation, x0, y0, x1, steps)
    
    def _solve(self):
        delta = self.delta
        equation = self.equation
        for x in self.grid[0][:-1]:
            y = self.grid[1][-1]

            k1 = delta*equation(x,y)
            k2 = delta*equation(x+delta/2,y+k1/2)
            k3 = delta*equation(x+delta/2,y+k2/2)
            k4 = delta*equation(x+delta,y+k3)

            self.grid[1] = np.append(self.grid[1], [y+(k1+2*k2+2*k3+k4)/6])

In [5]:
class Exact(Method):
    '''
    Exact initial value problem sollution
    '''
    
    def __init__(self, exact_solution, x0, y0, x1, steps):
        self.name = 'Exact Solution'
        super().__init__(exact_solution, x0, y0, x1, steps)
    
    def _solve(self):
        self.grid[1] = np.array([self.equation(x, self.grid[0][0], self.grid[1][0]) for x in self.grid[0]])

# Error counter

In [6]:
class LocalError(Method):
    '''
    Method for counting local error
    '''
    
    def __init__(self, exact, method):
        self.name = method.name + ' local error'
        self._solve(exact.grid, method.grid)
        
    def _solve(self, exact_grid, method_grid):
        self.grid = [exact_grid[0], np.abs(exact_grid[1]-method_grid[1])]

In [7]:
class GlobalError(Method):
    '''
    Method for counting global error
    '''
    
    def __init__(self, exact_solution, method, equation, x0, y0, x1):
        self.name = 'Global error ' + method.__name__
        self._solve(exact_solution, method, equation, x0, y0, x1)
        
        
    def _solve(self, exact_solution, method, equation, x0, y0, x1):
        grid_x = np.array([i*10 for i in range(2,50)])
        grid_y = []
        for step in grid_x:
            solution = method(equation, x0, y0, x1, step)
            exact = Exact(exact_solution, x0, y0, x1, step)
            local_error = LocalError(exact, solution)
            grid_y.append(np.max(local_error.grid[1]))
        grid_y = np.array(grid_y)
        self.grid = [grid_x, grid_y]
            

# GUI Backend

In [8]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

class GUI():
    
    def __init__(self, equation, exact_solution):
        self.equation = equation
        self.exact_solution = exact_solution
        
    def show(self):
        x0=widgets.IntSlider(value=1, min=1, max=100, step=1, description='x0', continuous_update=False)
        y0=widgets.IntSlider(value=3, min=1, max=100, step=1, description='y0', continuous_update=False)
        x1=widgets.IntSlider(value=10, min=1, max=100, step=1, description='X', continuous_update=False)
        steps=widgets.IntSlider(value=50, min=10, max=10000, step=10, description='steps', continuous_update=False)
        to_show=widgets.RadioButtons(
            options=['Solution',
                     'Local error',
                     'Global error'],
            description='To show',
            value='Solution',
        )

        euler = widgets.Checkbox(value=True, description='Euler')
        improved_euler = widgets.Checkbox(value=True, description='Improved Euler')
        runge_kutta = widgets.Checkbox(value=True, description='Runge Kutta')
        exact = widgets.Checkbox(value=False, description='Exact')

        def update_x1_range(*args):
            x1.min = x0.value + 1
            
        x0.observe(update_x1_range,'value')
        
        keys = {
                'x0': x0,
                'y0': y0,
                'x1': x1,
                'steps': steps,
                'exact': exact,
                'euler': euler,
                'improved_euler': improved_euler,
                'runge_kutta': runge_kutta,
                'to_show': to_show,
            }
        

        def update_visible(*args):
            if to_show.value != 'Solution':
                exact.layout.visibility = 'hidden'
            else:
                exact.layout.visibility = 'visible'
            if to_show.value == 'Global error':
                steps.layout.visibility = 'hidden'
            else:
                steps.layout.visibility = 'visible'

        to_show.observe(update_visible, 'value')

        args = widgets.VBox([x0, y0, x1, steps])
        methods = widgets.VBox([to_show, euler, improved_euler, runge_kutta, exact])
        
        ui = widgets.HBox([methods, args])
        
        out = widgets.interactive_output(self._show_manager, keys)
        
        display(ui, out)
        
    def _show_manager(self, to_show,
                     **kwargs):
        if to_show == 'Solution':
            self._show_equation(**kwargs)
        if to_show == 'Local error':
            del kwargs['exact']
            self._show_local_error(**kwargs)
        if to_show == 'Global error':
            del kwargs['steps']
            del kwargs['exact']
            self._show_global_error(**kwargs)
        
    def _show_equation(self,
                       x0, y0, x1, steps,
                       euler=True,
                       improved_euler=True,
                       runge_kutta=True,
                       exact=True
                      ):
            plt.rcParams['figure.figsize'] = [30, 30]
            plt.rcParams.update({'font.size': 22})
            if euler:
                Euler(self.equation, x0, y0, x1, steps).show(line='r-')
            if improved_euler:
                ImprovedEuler(self.equation, x0, y0, x1, steps).show(line='b-')
            if runge_kutta:
                RungeKutta(self.equation, x0, y0, x1, steps).show(line='g-')
            if exact:
                Exact(self.exact_solution, x0, y0, x1, steps).show(line='k--', linewidth=4)
            
    def _show_local_error(self,
                          x0, y0, x1, steps,
                          euler=True,
                          improved_euler=True,
                          runge_kutta=True,
                         ):
            plt.rcParams['figure.figsize'] = [30, 30]
            plt.rcParams.update({'font.size': 22})
            exact = Exact(self.exact_solution, x0, y0, x1, steps)
            if euler:
                method = Euler(self.equation, x0, y0, x1, steps)
                LocalError(exact, method).show(line='r-')
            if improved_euler:
                method = ImprovedEuler(self.equation, x0, y0, x1, steps)
                LocalError(exact, method).show(line='b-')
            if runge_kutta:
                method = RungeKutta(self.equation, x0, y0, x1, steps)
                LocalError(exact, method).show(line='g-')
                
    def _show_global_error(self,
                          x0, y0, x1,
                          euler=True,
                          improved_euler=True,
                          runge_kutta=True,
                         ):
            plt.rcParams['figure.figsize'] = [30, 30]
            plt.rcParams.update({'font.size': 22})
            if euler:
                GlobalError(self.exact_solution, Euler, self.equation, x0, y0, x1).show(line='r-')
            if improved_euler:
                GlobalError(self.exact_solution, ImprovedEuler, self.equation, x0, y0, x1).show(line='b-')
            if runge_kutta:
                GlobalError(self.exact_solution, RungeKutta, self.equation, x0, y0, x1).show(line='g-')

# Here comes my particular equation

In [131]:
def equation(x, y):
    return -x + y*(2*x+1)/x

def exact_solution(x, x0, y0):
    c = (y0-x0/2)/(x0*np.exp(2*x0))
    return c*np.exp(x*2)*x + x/2

In [137]:
gui = GUI(equation, exact_solution)
gui.show()

Output()